# Basic Structure of Training a Model

In [1]:
import tensorflow as tf
from datetime import datetime

## Hyper-parameters

In [2]:
num_classes = 10

epochs = 50
batch_size = 16

## Data pre-process

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

## Set up a model

In [4]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.flat = tf.keras.layers.Flatten(input_shape=(28, 28))
        self.d1 = tf.keras.layers.Dense(512, activation='relu')
        self.d2 = tf.keras.layers.Dense(256, activation='relu')
        self.d3 = tf.keras.layers.Dense(128, activation='relu')
        self.d4 = tf.keras.layers.Dense(64, activation='relu')
        self.d5 = tf.keras.layers.Dense(32, activation='relu')
        self.d6 = tf.keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        x = self.flat(inputs)
        x = self.d1(x)
        x = self.d2(x)
        x = self.d3(x)
        x = self.d4(x)
        x = self.d5(x)
        return self.d6(x)
    
model = MyModel()

## Set up callbacks
### checkpoint

In [5]:
checkpoint_path = 'my_checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                verbose=1)

### tensorboard

In [6]:
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

### reduce learning rate

In [7]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                 factor=0.1,
                                                 patience=3,
                                                 min_lr=0.00001)

## Set up optimizer, loss function, & metrics

In [8]:
optimizer = tf.keras.optimizers.RMSprop(0.001)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_acc = tf.keras.metrics.SparseCategoricalAccuracy()

## Compile the model

In [9]:
model.compile(optimizer, loss, metrics=[metric_acc])

## Fit the model

In [10]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    epochs=epochs,
    callbacks=[checkpoint, reduce_lr, tensorboard_callback],
)

Epoch 1/50
1866/1875 [============================>.] - ETA: 0s - loss: 1.5704 - sparse_categorical_accuracy: 0.8921
Epoch 00001: val_loss improved from inf to 1.53596, saving model to my_checkpoint.ckpt
1875/1875 [==============================] - 9s 5ms/step - loss: 1.5701 - sparse_categorical_accuracy: 0.8924 - val_loss: 1.5360 - val_sparse_categorical_accuracy: 0.9249 - lr: 0.0010
Epoch 2/50
1872/1875 [============================>.] - ETA: 0s - loss: 1.5230 - sparse_categorical_accuracy: 0.9380
Epoch 00002: val_loss improved from 1.53596 to 1.52525, saving model to my_checkpoint.ckpt
1875/1875 [==============================] - 11s 6ms/step - loss: 1.5229 - sparse_categorical_accuracy: 0.9380 - val_loss: 1.5253 - val_sparse_categorical_accuracy: 0.9357 - lr: 0.0010
Epoch 3/50
1872/1875 [============================>.] - ETA: 0s - loss: 1.5166 - sparse_categorical_accuracy: 0.9442
Epoch 00003: val_loss improved from 1.52525 to 1.51601, saving model to my_checkpoint.ckpt
1875/1875 [

1875/1875 [==============================] - ETA: 0s - loss: 1.4794 - sparse_categorical_accuracy: 0.9818
Epoch 00022: val_loss improved from 1.48767 to 1.48752, saving model to my_checkpoint.ckpt
1875/1875 [==============================] - 19s 10ms/step - loss: 1.4794 - sparse_categorical_accuracy: 0.9818 - val_loss: 1.4875 - val_sparse_categorical_accuracy: 0.9736 - lr: 1.0000e-05
Epoch 23/50
1873/1875 [============================>.] - ETA: 0s - loss: 1.4791 - sparse_categorical_accuracy: 0.9820
Epoch 00023: val_loss did not improve from 1.48752
1875/1875 [==============================] - 19s 10ms/step - loss: 1.4791 - sparse_categorical_accuracy: 0.9820 - val_loss: 1.4876 - val_sparse_categorical_accuracy: 0.9735 - lr: 1.0000e-05
Epoch 24/50
1873/1875 [============================>.] - ETA: 0s - loss: 1.4791 - sparse_categorical_accuracy: 0.9821
Epoch 00024: val_loss did not improve from 1.48752
1875/1875 [==============================] - 20s 11ms/step - loss: 1.4790 - sparse_ca

Epoch 45/50
1872/1875 [============================>.] - ETA: 0s - loss: 1.4783 - sparse_categorical_accuracy: 0.9828
Epoch 00045: val_loss did not improve from 1.48701
1875/1875 [==============================] - 20s 11ms/step - loss: 1.4783 - sparse_categorical_accuracy: 0.9828 - val_loss: 1.4873 - val_sparse_categorical_accuracy: 0.9738 - lr: 1.0000e-05
Epoch 46/50
1870/1875 [============================>.] - ETA: 0s - loss: 1.4783 - sparse_categorical_accuracy: 0.9828
Epoch 00046: val_loss did not improve from 1.48701
1875/1875 [==============================] - 22s 12ms/step - loss: 1.4783 - sparse_categorical_accuracy: 0.9828 - val_loss: 1.4873 - val_sparse_categorical_accuracy: 0.9739 - lr: 1.0000e-05
Epoch 47/50
1872/1875 [============================>.] - ETA: 0s - loss: 1.4783 - sparse_categorical_accuracy: 0.9828
Epoch 00047: val_loss did not improve from 1.48701
1875/1875 [==============================] - 22s 12ms/step - loss: 1.4783 - sparse_categorical_accuracy: 0.9829 -

## Check the training process via tensorboard

In [11]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 7257), started 0:19:34 ago. (Use '!kill 7257' to kill it.)

## Load the best weights

In [12]:
model.load_weights(checkpoint_path)

## Evaluate the model

In [15]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 814us/step - loss: 1.4870 - sparse_categorical_accuracy: 0.9742


[1.4870102405548096, 0.9742000102996826]